In [19]:
import pandas as pd
from pyspark.sql import functions as f
from pyspark.sql import *
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Create DF

In [2]:
df1 = {'idade':[16,17,18,20,22,24,21,16,14,np.NaN,26,49],
      'peso':[55,60,45,76,90,49,64,43,92,12,94,102],
      'sexo':['F','F','F','F','F','F','M','F','M','M','F','M']}

In [ ]:
#ou
from pyspark.ml.linalg import Vectors
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

In [21]:
df_pandas = pd.DataFrame(df1)
df_spark = spark.createDataFrame(df_pandas)

# Converter Pandas para Spark com Schema

In [ ]:
from pyspark.sql.types import *

# Auxiliar functions
def equivalent_type(f):
    if f == 'datetime64[ns]': return TimestampType()
    elif f == 'int64': return LongType()
    elif f == 'int32': return IntegerType()
    elif f == 'float64': return FloatType()
    else: return StringType()

def define_structure(string, format_type):
    try: typo = equivalent_type(format_type)
    except: typo = StringType()
    return StructField(string, typo)

# Given pandas dataframe, it will return a spark's dataframe.
def pandas_to_spark(pandas_df):
    columns = list(pandas_df.columns)
    types = list(pandas_df.dtypes)
    struct_list = []
    for column, typo in zip(columns, types): 
      struct_list.append(define_structure(column, typo))
    p_schema = StructType(struct_list)
    return sqlContext.createDataFrame(pandas_df, p_schema)

# .HEAD

In [22]:
df_pandas.head()

,idade,peso,sexo
0,16.0,55,F
1,17.0,60,F
2,18.0,45,F
3,20.0,76,F
4,22.0,90,F


In [23]:
df_spark.show(5)

+-----+----+----+
|idade|peso|sexo|
+-----+----+----+
| 16.0|  55|   F|
| 17.0|  60|   F|
| 18.0|  45|   F|
| 20.0|  76|   F|
| 22.0|  90|   F|
+-----+----+----+
only showing top 5 rows



# Select Column

In [24]:
df_pandas['idade']

0     16.0
1     17.0
2     18.0
3     20.0
4     22.0
5     24.0
6     21.0
7     16.0
8     14.0
9      NaN
10    26.0
11    49.0
Name: idade, dtype: float64

In [25]:
df_spark.select('idade').show()

+-----+
|idade|
+-----+
| 16.0|
| 17.0|
| 18.0|
| 20.0|
| 22.0|
| 24.0|
| 21.0|
| 16.0|
| 14.0|
|  NaN|
| 26.0|
| 49.0|
+-----+



# Boolean Filter

In [26]:
df_pandas[(df_pandas['idade']>20) & (df_pandas['peso']>45)]

,idade,peso,sexo
4,22.0,90,F
5,24.0,49,F
6,21.0,64,M
10,26.0,94,F
11,49.0,102,M


In [27]:
df_spark.filter((df_spark['idade']>20) & (df_spark['peso']>45)).show()

+-----+----+----+
|idade|peso|sexo|
+-----+----+----+
| 22.0|  90|   F|
| 24.0|  49|   F|
| 21.0|  64|   M|
| 26.0|  94|   F|
| 49.0| 102|   M|
+-----+----+----+



# New Column

In [28]:
df_pandas['idade2'] = df_pandas['idade'] + 10 
df_pandas

,idade,peso,sexo,idade2
0,16.0,55,F,26.0
1,17.0,60,F,27.0
2,18.0,45,F,28.0
3,20.0,76,F,30.0
4,22.0,90,F,32.0
5,24.0,49,F,34.0
6,21.0,64,M,31.0
7,16.0,43,F,26.0
8,14.0,92,M,24.0
9,NaN,12,M,NaN


In [29]:
df_spark = df_spark.withColumn('idade2',df_spark.idade+10)
df_spark.show()

+-----+----+----+------+
|idade|peso|sexo|idade2|
+-----+----+----+------+
| 16.0|  55|   F|  26.0|
| 17.0|  60|   F|  27.0|
| 18.0|  45|   F|  28.0|
| 20.0|  76|   F|  30.0|
| 22.0|  90|   F|  32.0|
| 24.0|  49|   F|  34.0|
| 21.0|  64|   M|  31.0|
| 16.0|  43|   F|  26.0|
| 14.0|  92|   M|  24.0|
|  NaN|  12|   M|   NaN|
| 26.0|  94|   F|  36.0|
| 49.0| 102|   M|  59.0|
+-----+----+----+------+



# Mean

In [30]:
df_pandas['idade'].mean()

22.09090909090909

In [31]:
df_spark.select(f.mean('idade')).show() # O spark não ignora o NaN

+----------+
|avg(idade)|
+----------+
|       NaN|
+----------+



# Dropna

In [32]:
df_pandas = df_pandas.dropna(subset=['idade'])
df_pandas

,idade,peso,sexo,idade2
0,16.0,55,F,26.0
1,17.0,60,F,27.0
2,18.0,45,F,28.0
3,20.0,76,F,30.0
4,22.0,90,F,32.0
5,24.0,49,F,34.0
6,21.0,64,M,31.0
7,16.0,43,F,26.0
8,14.0,92,M,24.0
10,26.0,94,F,36.0


In [33]:
df_spark = df_spark.dropna(subset=['idade'])
df_spark.show()

+-----+----+----+------+
|idade|peso|sexo|idade2|
+-----+----+----+------+
| 16.0|  55|   F|  26.0|
| 17.0|  60|   F|  27.0|
| 18.0|  45|   F|  28.0|
| 20.0|  76|   F|  30.0|
| 22.0|  90|   F|  32.0|
| 24.0|  49|   F|  34.0|
| 21.0|  64|   M|  31.0|
| 16.0|  43|   F|  26.0|
| 14.0|  92|   M|  24.0|
| 26.0|  94|   F|  36.0|
| 49.0| 102|   M|  59.0|
+-----+----+----+------+



In [34]:
df_pandas['idade'].mean()

22.09090909090909

In [35]:
df_spark.select(f.mean('idade')).show() # O spark não ignora o NaN

+-----------------+
|       avg(idade)|
+-----------------+
|22.09090909090909|
+-----------------+



# Isin

In [36]:
df_pandas[df_pandas.sexo.isin(['F'])]

,idade,peso,sexo,idade2
0,16.0,55,F,26.0
1,17.0,60,F,27.0
2,18.0,45,F,28.0
3,20.0,76,F,30.0
4,22.0,90,F,32.0
5,24.0,49,F,34.0
7,16.0,43,F,26.0
10,26.0,94,F,36.0


In [37]:
df_spark[df_spark.sexo.isin('F')].show() # Podemos usar como um booleano

+-----+----+----+------+
|idade|peso|sexo|idade2|
+-----+----+----+------+
| 16.0|  55|   F|  26.0|
| 17.0|  60|   F|  27.0|
| 18.0|  45|   F|  28.0|
| 20.0|  76|   F|  30.0|
| 22.0|  90|   F|  32.0|
| 24.0|  49|   F|  34.0|
| 16.0|  43|   F|  26.0|
| 26.0|  94|   F|  36.0|
+-----+----+----+------+



# New Column Com Função

In [38]:
def maior_idade(x):
    if x<18:
        z = str(x)+' menor de idade'
    else:
        z = str(x)+' maior de idade'
    return z

In [39]:
df_pandas['regularização'] = df_pandas['idade'].agg(maior_idade)
df_pandas

,idade,peso,sexo,idade2,regularização
0,16.0,55,F,26.0,16.0 menor de idade
1,17.0,60,F,27.0,17.0 menor de idade
2,18.0,45,F,28.0,18.0 maior de idade
3,20.0,76,F,30.0,20.0 maior de idade
4,22.0,90,F,32.0,22.0 maior de idade
5,24.0,49,F,34.0,24.0 maior de idade
6,21.0,64,M,31.0,21.0 maior de idade
7,16.0,43,F,26.0,16.0 menor de idade
8,14.0,92,M,24.0,14.0 menor de idade
10,26.0,94,F,36.0,26.0 maior de idade


## Necessário converter dados com UDF

In [40]:
# udf converte os valores de uma função para um data type do spark
from pyspark.sql import types
udf_maior_idade = f.udf(maior_idade,types.StringType())

In [41]:
df_spark.withColumn('regularização',udf_maior_idade('idade')).show()

+-----+----+----+------+-------------------+
|idade|peso|sexo|idade2|      regularização|
+-----+----+----+------+-------------------+
| 16.0|  55|   F|  26.0|16.0 menor de idade|
| 17.0|  60|   F|  27.0|17.0 menor de idade|
| 18.0|  45|   F|  28.0|18.0 maior de idade|
| 20.0|  76|   F|  30.0|20.0 maior de idade|
| 22.0|  90|   F|  32.0|22.0 maior de idade|
| 24.0|  49|   F|  34.0|24.0 maior de idade|
| 21.0|  64|   M|  31.0|21.0 maior de idade|
| 16.0|  43|   F|  26.0|16.0 menor de idade|
| 14.0|  92|   M|  24.0|14.0 menor de idade|
| 26.0|  94|   F|  36.0|26.0 maior de idade|
| 49.0| 102|   M|  59.0|49.0 maior de idade|
+-----+----+----+------+-------------------+



## Outro Modo

In [46]:
# lit executa algo em todas as linhas de uma coluna
# when cria um booleano para coluna
df_spark.withColumn('regularização', f.when((df_spark.idade<18),f.lit('menor de idade')).otherwise(f.lit('maior de idade'))).show()

+-----+----+----+------+--------------+
|idade|peso|sexo|idade2| regularização|
+-----+----+----+------+--------------+
| 16.0|  55|   F|  26.0|menor de idade|
| 17.0|  60|   F|  27.0|menor de idade|
| 18.0|  45|   F|  28.0|maior de idade|
| 20.0|  76|   F|  30.0|maior de idade|
| 22.0|  90|   F|  32.0|maior de idade|
| 24.0|  49|   F|  34.0|maior de idade|
| 21.0|  64|   M|  31.0|maior de idade|
| 16.0|  43|   F|  26.0|menor de idade|
| 14.0|  92|   M|  24.0|menor de idade|
| 26.0|  94|   F|  36.0|maior de idade|
| 49.0| 102|   M|  59.0|maior de idade|
+-----+----+----+------+--------------+



# Agg

In [47]:
df_pandas.agg({'peso':'max','sexo':'count'})

peso    102
sexo     11
dtype: int64

In [48]:
df_spark.agg({'peso':'max','sexo':'count'}).show()

+---------+-----------+
|max(peso)|count(sexo)|
+---------+-----------+
|      102|         11|
+---------+-----------+



# Info

In [49]:
df_pandas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 11
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   idade          11 non-null     float64
 1   peso           11 non-null     int64  
 2   sexo           11 non-null     object 
 3   idade2         11 non-null     float64
 4   regularização  11 non-null     object 
dtypes: float64(2), int64(1), object(2)
memory usage: 528.0+ bytes


In [50]:
df_spark.printSchema()

root
 |-- idade: double (nullable = true)
 |-- peso: long (nullable = true)
 |-- sexo: string (nullable = true)
 |-- idade2: double (nullable = true)



# Describe

In [51]:
df_pandas.describe()

,idade,peso,idade2
count,11.000000,11.000000,11.000000
mean,22.090909,70.000000,32.090909
std,9.648363,21.624061,9.648363
min,14.000000,43.000000,24.000000
25%,16.500000,52.000000,26.500000
50%,20.000000,64.000000,30.000000
75%,23.000000,91.000000,33.000000
max,49.000000,102.000000,59.000000


In [52]:
df_spark.describe().show()

+-------+-----------------+------------------+----+-----------------+
|summary|            idade|              peso|sexo|           idade2|
+-------+-----------------+------------------+----+-----------------+
|  count|               11|                11|  11|               11|
|   mean|22.09090909090909|              70.0|null|32.09090909090909|
| stddev|9.648363026488436|21.624060673240816|null|9.648363026488436|
|    min|             14.0|                43|   F|             24.0|
|    max|             49.0|               102|   M|             59.0|
+-------+-----------------+------------------+----+-----------------+



# Sort

In [53]:
df_pandas.sort_values(by='idade', ascending = False)

,idade,peso,sexo,idade2,regularização
11,49.0,102,M,59.0,49.0 maior de idade
10,26.0,94,F,36.0,26.0 maior de idade
5,24.0,49,F,34.0,24.0 maior de idade
4,22.0,90,F,32.0,22.0 maior de idade
6,21.0,64,M,31.0,21.0 maior de idade
3,20.0,76,F,30.0,20.0 maior de idade
2,18.0,45,F,28.0,18.0 maior de idade
1,17.0,60,F,27.0,17.0 menor de idade
0,16.0,55,F,26.0,16.0 menor de idade
7,16.0,43,F,26.0,16.0 menor de idade


In [54]:
df_spark.sort(df_spark.idade.desc()).show()

+-----+----+----+------+
|idade|peso|sexo|idade2|
+-----+----+----+------+
| 49.0| 102|   M|  59.0|
| 26.0|  94|   F|  36.0|
| 24.0|  49|   F|  34.0|
| 22.0|  90|   F|  32.0|
| 21.0|  64|   M|  31.0|
| 20.0|  76|   F|  30.0|
| 18.0|  45|   F|  28.0|
| 17.0|  60|   F|  27.0|
| 16.0|  55|   F|  26.0|
| 16.0|  43|   F|  26.0|
| 14.0|  92|   M|  24.0|
+-----+----+----+------+



# Value Counts

In [55]:
df_pandas['sexo'].value_counts()

F    8
M    3
Name: sexo, dtype: int64

In [56]:
df_spark.groupBy('sexo').count().show()

+----+-----+
|sexo|count|
+----+-----+
|   F|    8|
|   M|    3|
+----+-----+



# crosstab

In [57]:
pd.crosstab(index=df_pandas['idade'],columns = df_pandas['sexo'])

sexo,F,M
idade,,
14.0,0,1
16.0,2,0
17.0,1,0
18.0,1,0
20.0,1,0
21.0,0,1
22.0,1,0
24.0,1,0
26.0,1,0


In [58]:
df_spark.crosstab('idade','sexo').show()

+----------+---+---+
|idade_sexo|  F|  M|
+----------+---+---+
|      24.0|  1|  0|
|      14.0|  0|  1|
|      20.0|  1|  0|
|      21.0|  0|  1|
|      17.0|  1|  0|
|      22.0|  1|  0|
|      49.0|  0|  1|
|      18.0|  1|  0|
|      16.0|  2|  0|
|      26.0|  1|  0|
+----------+---+---+



# Criando uma coluna com ajuda de um dicionário

In [59]:
count = 0
rdd_ = df_spark.rdd.map(lambda x:x.asDict()).collect()  # Transforma cada rdd do df_spark em dicionário
for i in rdd_:                                          # Roda cada rdd do df
    i['id'] = count                                     # Adiciona o contador a cada coluna 
    count+=1
rdd_[0:2]

[{'idade': 16.0, 'peso': 55, 'sexo': 'F', 'idade2': 26.0, 'id': 0},
 {'idade': 17.0, 'peso': 60, 'sexo': 'F', 'idade2': 27.0, 'id': 1}]

In [60]:
df_spark_new = spark.createDataFrame(rdd_)            # Cria um DataFrame 

In [61]:
df_spark_new.show()

+---+-----+------+----+----+
| id|idade|idade2|peso|sexo|
+---+-----+------+----+----+
|  0| 16.0|  26.0|  55|   F|
|  1| 17.0|  27.0|  60|   F|
|  2| 18.0|  28.0|  45|   F|
|  3| 20.0|  30.0|  76|   F|
|  4| 22.0|  32.0|  90|   F|
|  5| 24.0|  34.0|  49|   F|
|  6| 21.0|  31.0|  64|   M|
|  7| 16.0|  26.0|  43|   F|
|  8| 14.0|  24.0|  92|   M|
|  9| 26.0|  36.0|  94|   F|
| 10| 49.0|  59.0| 102|   M|
+---+-----+------+----+----+



# Join = Merge

In [62]:
df_spark_new2 = spark.createDataFrame(pd.DataFrame({'id':[0,1,2,3,4,5,6,7,8,9,10], 'estado':['BA','BA','SP','SP','BA','SP','BA','RJ','RJ','RJ','BA']}))
df_spark_new2.show()

+---+------+
| id|estado|
+---+------+
|  0|    BA|
|  1|    BA|
|  2|    SP|
|  3|    SP|
|  4|    BA|
|  5|    SP|
|  6|    BA|
|  7|    RJ|
|  8|    RJ|
|  9|    RJ|
| 10|    BA|
+---+------+



In [63]:
df_spark_new.join(df_spark_new2,'id','left').show()

+---+-----+------+----+----+------+
| id|idade|idade2|peso|sexo|estado|
+---+-----+------+----+----+------+
|  0| 16.0|  26.0|  55|   F|    BA|
|  7| 16.0|  26.0|  43|   F|    RJ|
|  6| 21.0|  31.0|  64|   M|    BA|
|  9| 26.0|  36.0|  94|   F|    RJ|
|  5| 24.0|  34.0|  49|   F|    SP|
|  1| 17.0|  27.0|  60|   F|    BA|
| 10| 49.0|  59.0| 102|   M|    BA|
|  3| 20.0|  30.0|  76|   F|    SP|
|  8| 14.0|  24.0|  92|   M|    RJ|
|  2| 18.0|  28.0|  45|   F|    SP|
|  4| 22.0|  32.0|  90|   F|    BA|
+---+-----+------+----+----+------+



# PYspark com Pandas

In [ ]:
import pyspark.pandas as ps

In [ ]:
df = spark.read.json("examples/src/main/resources/people.json")

In [ ]:
df.createOrReplaceTempView("people")
sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

# CONTAINS
.like